In [1]:
import os
import pyspark

conf = pyspark.SparkConf()
conf = conf.setAppName("rl5083-midterm")
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040') ## to setup SPARK UI
conf = conf.set('spark.jars', os.environ['GRAPHFRAMES_PATH']) ## graphframes in spark configuration
sc = pyspark.SparkContext(conf=conf)
sc

24/11/09 02:18:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local[*] appName=rl5083-midterm>

In [2]:
spark = pyspark.SQLContext(sc)
spark

/opt/conda/envs/bigdata-spark/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


24/11/09 02:18:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Question 2: Language Models - 50 points

Discussion: If you have bigrams and trigrams, you can implement a simple Language Model that predicts the third word in a three-word sequence.

For example, to compute the probability P(times/new york), that is 'times' following the word 'new york', we use Maximum Likelihood Estime, i.e. MLE (ignoring out of vocabulary words, i.e. no smoothing)

Use MLE as follows:

$P(times | new~york) = \frac{count(trigrams(new~york~times))}{count(bigrams(new~york))}$

The input here is the same text input as homeworks 1 and 2: hw1text/*.

TO-DO: Compute the top 10 trigrams (count) for the input text, and show the conditional probability of the third word of each trigram.

In [3]:
import os
import zipfile
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col, count, regexp_replace, lower, explode, split, concat_ws

In [4]:
zip_file_path = 'hw1text.zip'
extraction_dir = 'hw1text/'

os.makedirs(extraction_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

extracted_files = os.listdir(extraction_dir)

text_files_path = [
    "20-01.txt",
    "20-02.txt",
    "20-03.txt",
    "20-04.txt",
    "20-05.txt"
]

text_files = [os.path.join(extraction_dir, file) for file in text_files_path]

In [5]:
df2 = spark.read.text(text_files)
df2_cleaned = df2.select(regexp_replace(lower(col("value")), "[^0-9a-z]+", " ").alias("cleaned_text"))
df2 = df2_cleaned.select(split(col("cleaned_text"), " ").alias("words"))

In [6]:
ngram = NGram(n=2, inputCol="words", outputCol="bigrams")
bigrams = ngram.transform(df2)
bigram_counts = bigrams.select(explode(col("bigrams")).alias("bigram")).groupBy("bigram").agg(count("*").alias("bigram_count"))

In [7]:
ngram = NGram(n=3, inputCol="words", outputCol="trigrams")
trigrams = ngram.transform(df2)
trigram_counts = trigrams.select(explode(col("trigrams")).alias("trigram")).groupBy("trigram").agg(count("*").alias("trigram_count"))

In [8]:
trigram_counts = trigram_counts.withColumn("bigram", split(col("trigram"), " ").getItem(0)).withColumn("bigram", concat_ws(" ", col("bigram"), split(col("trigram"), " ").getItem(1)))

conditional_probabilities = trigram_counts.join(bigram_counts, "bigram").withColumn("conditional_probability", col("trigram_count") / col("bigram_count")).select("trigram", "trigram_count", "bigram", "bigram_count", "conditional_probability").orderBy(col("trigram_count").desc()).limit(10)
conditional_probabilities.show(truncate=False)

+------------------+-------------+----------+------------+-----------------------+
|trigram           |trigram_count|bigram    |bigram_count|conditional_probability|
+------------------+-------------+----------+------------+-----------------------+
|lt p gt           |1928         |lt p      |1930        |0.9989637305699481     |
|the covid 19      |1718         |the covid |1760        |0.9761363636363637     |
|do n t            |1662         |do n      |1662        |1.0                    |
|of covid 19       |1589         |of covid  |1621        |0.9802590993214065     |
|the spread of     |1196         |the spread|1306        |0.9157733537519143     |
|p gt lt           |1037         |p gt      |1936        |0.5356404958677686     |
|the number of     |1037         |the number|1103        |0.9401631912964642     |
|gt lt p           |1023         |gt lt     |1792        |0.5708705357142857     |
|one of the        |953          |one of    |1491        |0.6391683433936955     |
|of 

# Question 3: Ranking over Partitions - 50 points

Compute the top 3 items sold per daypart for the problem in Homework 2, Question 1. (Bakery.csv)

Daypart = 
- morning if time >= 6AM, < 11AM
- noon if time >= 11AM, < 2PM
- afternoon if time >= 2PM, < 5PM
- evening if time >= 5PM, <6AM 

The format MUST be in the format (example) and show ALL rows:     
morning coffe, bagel, pastry      
noon soda, ham, potatoes   
etc.

In [9]:
from pyspark.sql.functions import when, rank, collect_list
from pyspark.sql import Window

In [10]:
df3 = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/Bakery.csv") 
df3 = df3.withColumn("Daypart",
                when((col("Time").between("06:00:00", "10:59:59")), "morning")
                .when((col("Time").between("11:00:00", "13:59:59")), "noon")
                .when((col("Time").between("14:00:00", "16:59:59")), "afternoon")
                .otherwise("evening"))

item_counts = df3.groupBy("Daypart", "Item").agg(count("Transaction").alias("qty"))
top_items = item_counts.withColumn("rank", rank().over(Window.partitionBy("Daypart").orderBy(col("qty").desc()))).filter(col("rank") <= 3)
top_items = top_items.groupBy("Daypart").agg(concat_ws(", ", collect_list("Item")).alias("Top-3 Items"))

print("Top-3 (by qty) items bought by Daypart:")
top_items.show()

Top-3 (by qty) items bought by Daypart:


+---------+--------------------+
|  Daypart|         Top-3 Items|
+---------+--------------------+
|afternoon|  Coffee, Bread, Tea|
|  evening|  Coffee, Bread, Tea|
|  morning|Coffee, Bread, Pa...|
|     noon|  Coffee, Bread, Tea|
+---------+--------------------+



# Question 4: Duplicate Detection with Minhash – 50 points

Minhash/LSH is an algorithm based on cryptographic hashes that computes similarity between a pair of entities. It is heavily used in Web Search and product similarity applications.

Details of the algorithm can be found in Chapter 3 of Mining Massive Datasets.    
http://infolab.stanford.edu/~ullman/mmds/ch3.pdf

For example, assume I teach a course at NYU and I trust the students to learn the material by doing original work. However, I think I see too many cases of plagiarism and copying. I could use this algorithm to easily detect near-duplicates and copying so you I can fail those students. 

__Problem Statement__

For this problem, let’s use the similarity algorithm to find the most similar items. Assume I you are reading news items in the HuffPost website (https://www.huffpost.com/) and last clicked on a new article about gun control in the USA. The next day, I return to the website and it decides to prioritize new items for me based on the last news item I read. 

__The Data__:

In Jupyterhub: `shared/data/huffpost.json`
Base news item short description "Kitten Born With Twisted Arms And Legs Finds A Mom Who Knows She\u2019s Perfect"

__TODO__

Use the Minhash/LSH algorithm to find URL link, headline, category, and short description of the 5 most similar items to the Item above (based on the "short_description" field). The algorithm relies on the concept of distances to define similarity. For this exercise, use Jaccard similarity. 

NOTE: You can use any a third party Python library like 'datasketch'. (http://ekzhu.com/datasketch/minhash.html). Datasketch is installed in the JupyterHub environment for this class. Note, SparkML also has a Minhash/LSH library.

In [11]:
!pip install datasketch

Defaulting to user installation because normal site-packages is not writeable


In [12]:
from datasketch import MinHash
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType

In [13]:
def get_minhash(short_desc):
    m = MinHash()
    for word in short_desc.split():
        m.update(word.encode('utf8'))
    return m

In [14]:
def compute_jaccard(minhash1, minhash2):
    return minhash1.jaccard(minhash2)

In [15]:
df4 = spark.read.json('shared/data/Huffpost.json')
df4.printSchema()

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)



In [16]:
minhash_udf = udf(lambda desc: get_minhash(desc), StringType())

In [17]:
df4 = df4.withColumn("minhash", minhash_udf(col("short_description")))
df4.printSchema()

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- minhash: string (nullable = true)



In [18]:
base_short_desc = "Kitten Born With Twisted Arms And Legs Finds A Mom Who Knows She\u2019s Perfect"
base_minhash = get_minhash(base_short_desc)

In [19]:
jaccard_udf = udf(lambda minhash: compute_jaccard(base_minhash, minhash), FloatType())

df4 = df4.withColumn("Estimated Jaccard", jaccard_udf(col("minhash")))
df4 = df4.orderBy(col("Estimated Jaccard").desc()).limit(5)
df4.select("link", "headline", "category", "short_description", "Estimated Jaccard").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------+
|link                                                                                                                               |headline                                                                            |category |short_description                                                                                                              |Estimated Jaccard|
+-----------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+---------+----------------------